In [11]:
%load_ext autoreload
%autoreload 2
from davidyu_cfg import *
from functions.baostock.stock_return import *
from functions.common.loadModule.load_module_kdj import *
import matplotlib.pyplot as plt



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
def new_kdj(df,kdj_day=9,k_stat=3,d_stat=3):
    low_list = df['low'].rolling(kdj_day, min_periods=kdj_day).min()
    low_list.fillna(value = df['low'].expanding().min(), inplace = True)
    high_list = df['high'].rolling(kdj_day, min_periods=kdj_day).max()
    high_list.fillna(value = df['high'].expanding().max(), inplace = True)

    rsv = (df['close'] - low_list) / (high_list - low_list) * 100
    df['K'] = pd.DataFrame(rsv).ewm(com=k_stat-1).mean()
    df['D'] = df['K'].ewm(com=d_stat-1).mean()
    df['J'] = 3 * df['K'] - 2 * df['D']
    return df
def new_macd(df2,col,MACD_EMA_SHORT = 12,MACD_EMA_LONG = 26, MACD_EMA_SIGNAL = 9):
    fast=df2[col].ewm(
                ignore_na=False, span=MACD_EMA_SHORT,
                min_periods=0, adjust=True).mean()
    slow=df2[col].ewm(
                ignore_na=False, span=MACD_EMA_LONG,
                min_periods=0, adjust=True).mean()
    # df2["volume_26"].plot(secondary_y=True, style='r*-')
    # df2["volume_12"].plot(secondary_y=True, style='b*-')
    df2['macd'] = fast-slow
    df2['macds'] = df2["macd"].ewm(
                ignore_na=False, span=MACD_EMA_SIGNAL,
                min_periods=0, adjust=True).mean()
    df2['macdh'] = (df2['macd'] - df2['macds'])
    return df2

In [16]:

# df

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [18]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
# df2 = df1[df1["dt"]>="2020-03-01"]
from plotly.subplots import make_subplots


def plot_new_kdj(df2):
    df2 = df
    layout = dict(title_x=0.5, # title居中
              font=dict(family="Times New Roman",size=20,color="RebeccaPurple"),
              # 字体设置
              coloraxis_colorbar=dict(xanchor="left",x=0.75,ticks="outside"),
              # 颜色条设置
              margin=dict(b= 0,l=20, r=0, t= 0),
              width=1000,
              height=800
              # 大小设置
             )
    fig_all = make_subplots(rows=2,  # 将画布分为两行
            cols=1,  # 将画布分为两列
            subplot_titles=["", 
                            ""
                         ],  # 子图的标题
            x_title="",
            y_title="y轴标题", shared_xaxes=True
           )

    trace0 = go.Scatter(
        name="close",
        x=df2["dt"], y=df2["close"],mode='markers+lines'
    )

    trace2 = go.Scatter(
         name="K",
         x=df2["dt"], y=df2["K"],mode='markers+lines'
    )
    trace3 = go.Scatter(
         name="D",
         x=df2["dt"], y=df2["D"],mode='markers+lines'
    )

    trace4 = go.Scatter(
         name="J",
         x=df2["dt"], y=df2["J"],mode='markers+lines'
    )

    a1=tuple([trace2,trace3,trace4])
    fig_all.append_trace(trace0,1,1)  # 将trace0添加到第一行第一列的位置
    fig_all.add_traces(a1,2,1)

    fig_all['layout']['yaxis1'].update(domain=[0.5, 1])
    fig_all['layout']['yaxis2'].update(domain=[0, 0.46])

    fig_all.update_layout(layout)
    fig_all.update_layout(xaxis1_rangeslider_visible=False)


    fig_all.show()
# fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
# df2 = df1[df1["dt"]>="2020-03-01"]
from plotly.subplots import make_subplots

def plot_new_macd(df3):
    layout = dict(title_x=0.5, # title居中
              font=dict(family="Times New Roman",size=20,color="RebeccaPurple"),
              # 字体设置
              coloraxis_colorbar=dict(xanchor="left",x=0.75,ticks="outside"),
              # 颜色条设置
              margin=dict(b= 0,l=20, r=0, t= 0),
              width=1000,
              height=800
              # 大小设置
             )
    fig_all = make_subplots(rows=4,  # 将画布分为两行
            cols=1,  # 将画布分为两列
            subplot_titles=["", 
                            ""
                         ],  # 子图的标题
            x_title="",
            y_title="y轴标题", shared_xaxes=True
           )

    trace0 = go.Scatter(
        name="close",
        x=df3["dt"], y=df3["close"],mode='markers+lines'
    )
    trace1 = go.Bar(
        name="macd_close",
        x=df3["dt"], y=df3["macd_close"]
    )

    trace2 = go.Bar(
         name="macd_high",
         x=df3["dt"], y=df3["macd_high"],
    )
    trace3 = go.Bar(
         name="macd_low",
         x=df3["dt"], y=df3["macd_low"],
    )

    # fig = px.bar(df2, x='dt', y="macdh_volume")

    fig_all.append_trace(trace0,1,1)  # 将trace0添加到第一行第一列的位置
    fig_all.append_trace(trace1,2,1)  # 将trace0添加到第一行第一列的位置
    fig_all.append_trace(trace2,3,1)  # 将trace0添加到第一行第一列的位置
    fig_all.append_trace(trace3,4,1)  # 将trace0添加到第一行第一列的位置

    fig_all['layout']['yaxis1'].update(domain=[0.8, 1])
    fig_all['layout']['yaxis2'].update(domain=[0.6, 0.75])
    fig_all['layout']['yaxis3'].update(domain=[0.4, 0.55])
    fig_all['layout']['yaxis4'].update(domain=[0.1, 0.35])



    fig_all.update_layout(layout)
    fig_all.update_layout(xaxis1_rangeslider_visible=False)
    fig_all.update_layout(xaxis2_rangeslider_visible=False)
    fig_all.update_layout(xaxis4_rangeslider_visible=True)

    fig_all.show()
# fig.show()

In [23]:
stock_index="600547"
start_date = "2020-11-01"
file_name=os.path.join("/home/davidyu/stock/data/history_data/dfcf_fuquan/stock_index",stock_index+"_fuquan.csv")
df1 = pd.read_csv(file_name)
df2 = df1[df1["dt"]>=start_date]
df = df2.copy()
kdj_day = 6
k_stat = 3
d_stat = 3
j_stat = 2
df = new_kdj(df,kdj_day,k_stat,d_stat)  # 卖点
plot_new_kdj(df2)

In [30]:
stock_index="600547"
start_date = "2020-12-01"
file_name=os.path.join("/home/davidyu/stock/data/history_data/dfcf_fuquan/stock_index",stock_index+"_fuquan.csv")
df1 = pd.read_csv(file_name)
df2 = df1[df1["dt"]>=start_date]
df = df2.copy()
kdj_day = 24
k_stat = 6
d_stat = 3
j_stat = 2
df = new_kdj(df,kdj_day,k_stat,d_stat)  # 卖点
plot_new_kdj(df2)

In [75]:
stock_index="000917"
start_date = "2020-05-01"
file_name=os.path.join("/home/davidyu/stock/data/history_data/dfcf_fuquan/stock_index",stock_index+"_fuquan.csv")
df1 = pd.read_csv(file_name)
df2 = df1[df1["dt"]>=start_date]
# df = df2.copy()
# kdj_day = 24
# k_stat = 6
# d_stat = 3
# j_stat = 2
# df = new_kdj(df,kdj_day,k_stat,d_stat)  # 卖点



MACD_EMA_SHORT = 5
MACD_EMA_LONG = 34
MACD_EMA_SIGNAL = 5
df3=df2.copy()
df3 = new_macd(df3,"close",MACD_EMA_SHORT,MACD_EMA_LONG,MACD_EMA_SIGNAL)
df3["macd_close"] = df3["macdh"]
df3 = new_macd(df3,"high",MACD_EMA_SHORT,MACD_EMA_LONG,MACD_EMA_SIGNAL)
df3["macd_high"] = df3["macdh"]
df3 = new_macd(df3,"low",MACD_EMA_SHORT,MACD_EMA_LONG,MACD_EMA_SIGNAL)
df3["macd_low"] = df3["macdh"]

plot_new_macd(df3)

In [55]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
# df2 = df1[df1["dt"]>="2020-03-01"]
from plotly.subplots import make_subplots

def plot_new_macd(df3):
    layout = dict(title_x=0.5, # title居中
              font=dict(family="Times New Roman",size=20,color="RebeccaPurple"),
              # 字体设置
              coloraxis_colorbar=dict(xanchor="left",x=0.75,ticks="outside"),
              # 颜色条设置
              margin=dict(b= 0,l=20, r=0, t= 0),
              width=1000,
              height=800
              # 大小设置
             )
    fig_all = make_subplots(rows=4,  # 将画布分为两行
            cols=1,  # 将画布分为两列
            subplot_titles=["", 
                            ""
                         ],  # 子图的标题
            x_title="",
            y_title="y轴标题", shared_xaxes=True
           )

    trace0 = go.Scatter(
        name="close",
        x=df3["dt"], y=df3["close"],mode='markers+lines'
    )
    trace1 = go.Bar(
        name="macd_close",
        x=df3["dt"], y=df3["macd_close"]
    )

    trace2 = go.Bar(
         name="macd_high",
         x=df3["dt"], y=df3["macd_high"],
    )
    trace3 = go.Bar(
         name="macd_low",
         x=df3["dt"], y=df3["macd_low"],
    )

    # fig = px.bar(df2, x='dt', y="macdh_volume")

    fig_all.append_trace(trace0,1,1)  # 将trace0添加到第一行第一列的位置
    fig_all.append_trace(trace1,2,1)  # 将trace0添加到第一行第一列的位置
    fig_all.append_trace(trace2,3,1)  # 将trace0添加到第一行第一列的位置
    fig_all.append_trace(trace3,4,1)  # 将trace0添加到第一行第一列的位置

    fig_all['layout']['yaxis1'].update(domain=[0.8, 1])
    fig_all['layout']['yaxis2'].update(domain=[0.6, 0.75])
    fig_all['layout']['yaxis3'].update(domain=[0.4, 0.55])
    fig_all['layout']['yaxis4'].update(domain=[0.1, 0.35])



    fig_all.update_layout(layout)
    fig_all.update_layout(xaxis1_rangeslider_visible=False)
    fig_all.update_layout(xaxis2_rangeslider_visible=False)
    fig_all.update_layout(xaxis4_rangeslider_visible=True)

    fig_all.show()
# fig.show()